<a href="https://colab.research.google.com/github/danielvijill/solar_energy/blob/main/geosite_temperature_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
folder_path = '/content/drive/MyDrive/Colab Notebooks'

In [5]:
csv_file_path = f'{folder_path}/sites.csv'
df = pd.read_csv(csv_file_path)
print(df.head())

  ID  Nombre                Cuadrante Departamento Coordenadas UTM Grid  \
0  1     NaN                ARAMECINA    F MORAZAN                44/11   
1  2     NaN                ARAMECINA    F MORAZAN                45/14   
2  3     NaN                   ARENAL         YORO              100/051   
3  4     NaN  AZACUALPA-RIO GUAYAMBRE      OLANCHO                99/00   
4  5     NaN  AZACUALPA-RIO GUAYAMBRE      OLANCHO                00/99   

   Latitud  Longitud  Temperatura Min  Temperatura Max  
0   13.670   -87.509              NaN              NaN  
1   13.707   -87.507              NaN              NaN  
2   15.422   -86.883             61.0             67.0  
3   14.546   -85.992              NaN              NaN  
4   14.505   -85.971              NaN              NaN  


In [20]:
# Filter the DataFrame into two sets: one with known temperatures and one with unknown temperatures
df_known = df.dropna(subset=['Temperatura Max'])
df_known = df_known.dropna(subset=['Latitud'])
df_known = df_known.dropna(subset=['Longitud'])
df_unknown = df[df['Temperatura Max'].isna()]

# Define the features (input) and target (output) for the known data
X_train = df_known[['Latitud', 'Longitud']].values
y_train = df_known['Temperatura Max'].values

# Define the features (input) for the unknown data (we will predict the target for these)
X_predict = df_unknown[['Latitud', 'Longitud']].values

In [23]:
# Create a simple TensorFlow model (a feedforward neural network)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1) # Output layer for temperature prediction
])

# Compile the model
model.compile(optimizer='adam', loss='mse') # Using Mean Squared Error as loss

# Train the model
model.fit(X_train, y_train, epochs=100, verbose=0)

# Predict the temperatures for the unknown data
predicted_temperatures = model.predict(X_predict)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [24]:
# Create a copy of the original DataFrame
df_predicted = df.copy()

# Update the 'Temperatura Max' column in the copied DataFrame with the predicted values
# Ensure that the indexing aligns between df_unknown and the predicted_temperatures
df_predicted.loc[df_unknown.index, 'Temperatura Max'] = predicted_temperatures.flatten()

# Print the updated DataFrame head to see the estimated temperatures
print("\nDataFrame with estimated temperatures:")
print(df_predicted.head())


DataFrame with estimated temperatures:
  ID  Nombre                Cuadrante Departamento Coordenadas UTM Grid  \
0  1     NaN                ARAMECINA    F MORAZAN                44/11   
1  2     NaN                ARAMECINA    F MORAZAN                45/14   
2  3     NaN                   ARENAL         YORO              100/051   
3  4     NaN  AZACUALPA-RIO GUAYAMBRE      OLANCHO                99/00   
4  5     NaN  AZACUALPA-RIO GUAYAMBRE      OLANCHO                00/99   

   Latitud  Longitud  Temperatura Min  Temperatura Max  
0   13.670   -87.509              NaN        56.418564  
1   13.707   -87.507              NaN        56.425591  
2   15.422   -86.883             61.0        67.000000  
3   14.546   -85.992              NaN        55.691650  
4   14.505   -85.971              NaN        55.669754  


In [21]:
# print(np.isnan(X_train).sum(), np.isinf(X_train).sum())
# print(np.isnan(y_train).sum(), np.isinf(y_train).sum())

0 0
0 0


In [22]:
# print(np.isnan(X_predict).sum(), np.isinf(X_predict).sum())

34 0


In [26]:
df_predicted.to_csv('predicted_temperatures.csv', index=False)